In [1]:
import pandas as pd 
import seaborn as sn 
import matplotlib.pyplot as plt 
import numpy as np 
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

### Loading Dataset 

In [2]:
df_full_feature = pd.read_csv('../Dataset/Normalisasi/processed_dataset_10_norm.csv',sep=',')
df_chi_square = pd.read_csv('../Dataset/Normalisasi/processed_dataset_chi_square_norm.csv', sep=',')
df_feature_selection = pd.read_csv('../Dataset/Normalisasi/processed_dataset_5_norm.csv',sep=',')
df_slice = pd.read_csv('../Dataset/Normalisasi/processed_dataset_slice_norm.csv',sep=',')

### Splitting Variable

In [3]:
# Dataset Full Feature
full_feature_x = df_full_feature[df_full_feature.columns[:9]]
full_feature_y = df_full_feature['label']

full_feature_x.head(5)

,src,dst,Protocol,pktrate,pktperflow,dur,dur_nsec,bytecount,pktcount
0,0.055556,0.647059,0.0,0.000000,0.001511,0.522594,0.235235,0.000640,0.003696
1,0.333333,0.705882,0.5,0.799687,0.799583,0.204147,0.114114,0.081254,0.851453
2,0.555556,0.882353,0.0,0.001565,0.001563,0.436470,0.243243,0.000534,0.003081
3,0.833333,0.941176,0.5,0.516432,0.516884,0.052100,0.622623,0.014585,0.125043
4,0.111111,0.647059,1.0,0.428795,0.428504,0.152578,0.579580,0.618070,0.335646


In [4]:
# Dataset Chi Square 
chi_square_x = df_chi_square[df_chi_square.columns[:5]]
chi_square_y = df_chi_square['label']

chi_square_x.head(5)

,dst,src,Protocol,pktcount,pktcount.1
0,0.647059,0.055556,0.0,0.003696,0.003696
1,0.705882,0.333333,0.5,0.851453,0.851453
2,0.882353,0.555556,0.0,0.003081,0.003081
3,0.941176,0.833333,0.5,0.125043,0.125043
4,0.647059,0.111111,1.0,0.335646,0.335646


In [5]:
# Dataset Feature Selection
feature_selection_x = df_feature_selection[df_feature_selection.columns[:5]]
feature_selection_y = df_feature_selection['label']

feature_selection_x.head(5)

,pktrate,pktperflow,Protocol,src,dst
0,0.000000,0.001511,0.0,0.055556,0.647059
1,0.799687,0.799583,0.5,0.333333,0.705882
2,0.001565,0.001563,0.0,0.555556,0.882353
3,0.516432,0.516884,0.5,0.833333,0.941176
4,0.428795,0.428504,1.0,0.111111,0.647059


In [6]:
# Dataset Slice
slice_x = df_slice[df_slice.columns[:4]]
slice_y= df_slice['label']

slice_x.head(5)

,dst,src,Protocol,bytecount
0,0.647059,0.055556,0.0,0.000640
1,0.705882,0.333333,0.5,0.081254
2,0.882353,0.555556,0.0,0.000534
3,0.941176,0.833333,0.5,0.014585
4,0.647059,0.111111,1.0,0.618070


### Full Model 

In [10]:
#Splitting Data 

x_train,x_test,y_train,y_test = train_test_split(full_feature_x,full_feature_y,test_size=0.30,random_state=9)

x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_train.shape


(50540, 9, 1)

In [8]:
model_full = Sequential()
model_full.add(LSTM(100, input_shape=(x_train.shape[1],1), return_sequences=True))
model_full.add(Dropout(0.5))
model_full.add(LSTM(32, return_sequences=False))
model_full.add(Dense(10, activation='relu'))
model_full.add(Dense(3, activation='softmax'))

model_full.compile(optimizer="adam",loss="sparse_categorical_crossentropy", metrics=['accuracy','MeanSquaredError', 'MeanAbsoluteError'])
model_full.fit(x_train, y_train, epochs=10)


Epoch 1/10
1580/1580 [==============================] - 94s 54ms/step - loss: 0.5998 - accuracy: 0.6709 - mean_squared_error: 0.3263 - mean_absolute_error: 0.4635A: 1: - ETA: 12s - loss: 0.6163 - accuracy: 0.65 - ETA: 5s
Epoch 2/10
1580/1580 [==============================] - 56s 35ms/step - loss: 0.2367 - accuracy: 0.9037 - mean_squared_error: 0.4128 - mean_absolute_error: 0.4637A: 2:14 - l - ETA: 50s - loss: 0.2553 - accuracy: 0.8977 - mean_squared_error: 0.4071 - - ETA: 48s - loss: 0.2544 - accuracy: 0.8980 - mean_squared_error - ETA: 37s - loss: 0.2503 - accuracy: 0.8995 - mean_squared_error: 0.4 - ETA: 7s - loss: 0.2397 - accuracy: 0.9028 - mean_squared_error: 0.4119 - mean_absolute - ETA: 6s - loss: 0.2391 - accuracy: 0.903
Epoch 3/10
1580/1580 [==============================] - 33s 21ms/step - loss: 0.1902 - accuracy: 0.9163 - mean_squared_error: 0.4252 - mean_absolute_error: 0.4634
Epoch 4/10
1580/1580 [==============================] - 48s 30ms/step - loss: 0.1773 - accuracy: 

In [11]:
y_pred_train = model_full.predict(x_train)
y_pred_test = model_full.predict(x_test)

In [12]:
import warnings
warnings.filterwarnings('ignore')
y_pred_train = [np.argmax(element) for element in y_pred_train]
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))
print("Accuracy:",accuracy_score(y_train, y_pred_train)*100)


[[29027  1762]
 [ 1010 18741]]
              precision    recall  f1-score   support

           0       0.97      0.94      0.95     30789
           1       0.91      0.95      0.93     19751

    accuracy                           0.95     50540
   macro avg       0.94      0.95      0.94     50540
weighted avg       0.95      0.95      0.95     50540

Accuracy: 94.51523545706371


In [13]:
warnings.filterwarnings('ignore')
y_pred_test = [np.argmax(element) for element in y_pred_test]
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))
print("Accuracy:",accuracy_score(y_test, y_pred_test)*100)

[[12444   786]
 [  450  7981]]
              precision    recall  f1-score   support

           0       0.97      0.94      0.95     13230
           1       0.91      0.95      0.93      8431

    accuracy                           0.94     21661
   macro avg       0.94      0.94      0.94     21661
weighted avg       0.94      0.94      0.94     21661

Accuracy: 94.29389224874198


Belajar Lagi Mengenai Komponen LSTM dan Kemudian Mengenai Konsep Reshaping Dari LSTM